I have a hunch that there's going to be some difficulties with nesting the blocks into the precincts, so I'm going to use the maup disaggregation and proration features to hopefully combat that. 

In [1]:
import geopandas
import maup
import warnings

# turn off annoying geoseries isna warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
#enable progres bar
maup.progress.enabled = True


# Given blocks and precincts, first split blocks so that they nest into precincts, 
# prorating by population, then aggregate into precincts. 
# blocks_f_name = str, file name of blocks shp file
# precincts_f_name = str, file name of precincts shp file
# out_f_name = str, file name to write final precincts shp file to (must end in .shp)
def blocks_to_precincts_disag(blocks_f_name, precincts_f_name, out_f_name):
    # read in files
    blocks = geopandas.read_file(blocks_f_name)
    precincts = geopandas.read_file(precincts_f_name)

    # change crs of shp files to be projected (using VA north)
    blocks = blocks.to_crs(epsg=2283)
    precincts = precincts.to_crs(epsg=2283)
    # remove any bowties (little imperfections in the polygons)
    blocks.geometry = blocks.buffer(0)
    precincts.geometry = precincts.buffer(0)
    # reindex
    blocks = blocks.reset_index(drop = True)
    precincts = precincts.reset_index(drop = True)

    columns = ["pop", "TotPop", "BlackPop", "HispPop", "VAP", "BlackVAP", "HISPVAP"]
    # Include area_cutoff=0 to ignore any intersections with no area,
    # like boundary intersections, which we do not want to include in
    # our proration.
    pieces = maup.intersections(blocks, precincts, area_cutoff=0)

    # Weight by prorated population from blocks THIS TAKES 3 HOURS
    assign = maup.assign(blocks, pieces)
    weights = blocks["pop"].groupby(assign).sum()

    # Normalize the weights so that votes are allocated according to their
    # share of population in the old_precincts
    weights = maup.normalize(weights, level=0)

    # Use blocks to estimate population of each piece
    precincts[columns] = maup.prorate(
        pieces,
        blocks[columns],
        weights=weights
    )
    # write to file
    precincts.to_file(out_f_name)

In [3]:
blocks_f_name = "zip://C:/Users/madie/OneDrive/data/ipums/VA_block_2017_data.zip"
precincts_f_name = "zip://C:/Users/madie/OneDrive/data/vest/va_2017_statehouse.zip"
out_f_name = "C:/Users/madie/OneDrive/data/pre-redist/VA_precinct_2017/VA_precinct_2017.shp"
blocks_to_precincts_disag(blocks_f_name, precincts_f_name, out_f_name)

100%|█████████████████████████████████████████████████████████████████████████| 379448/379448 [51:33<00:00, 122.67it/s]


In [4]:
blocks_f_name = "zip://C:/Users/madie/OneDrive/data/ipums/VA_block_2019_data.zip"
precincts_f_name = "zip://C:/Users/madie/OneDrive/data/vest/va_2019_statehouse.zip"
out_f_name = "C:/Users/madie/OneDrive/data/pre-redist/VA_precinct_2019/VA_precinct_2019.shp"
blocks_to_precincts_disag(blocks_f_name, precincts_f_name, out_f_name)

100%|█████████████████████████████████████████████████████████████████████████| 579281/579281 [23:26<00:00, 411.85it/s]
